In [22]:
# # Install if you have never used these: unblock the lines below to install if needed

# !pip install webdriver-manager
# !pip3 install lxml
# !pip3 install selenium
# !pip3 install webdriver_manager
# !pip install --upgrade pip
# !pip install -U selenium
# !pip install msedge-selenium-tools


     ---------------------------------------- 0.0/904.6 kB ? eta -:--:--
     --- ----------------------------------- 92.2/904.6 kB 1.7 MB/s eta 0:00:01
     ---------------- --------------------- 389.1/904.6 kB 4.0 MB/s eta 0:00:01
     -------------------- ----------------- 491.5/904.6 kB 3.9 MB/s eta 0:00:01
     ------------------------------ ------- 737.3/904.6 kB 3.9 MB/s eta 0:00:01
     -------------------------------------- 904.6/904.6 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.16.0
    Uninstalling selenium-4.16.0:
      Successfully uninstalled selenium-4.16.0



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
# --------- import necessary modules -------

# For webscraping
from bs4 import BeautifulSoup

# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait

In [24]:
import selenium

# Check version I am running
selenium.__version__

'4.15.2'

In [25]:
# Selenium 4:

from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [26]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [28]:
# Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
option= webdriver.ChromeOptions()

# Going undercover:
option.add_argument("--incognito")


# # Consider this if the application works and you know how it works for speed ups and rendering!

# option.add_argument('--headless=chrome')

In [31]:
# Define job search keyword for Data Analyst
job_search_keyword = ['Data+Analyst']

# Finding position, radius=35 miles, sort by date and starting page
pagination_url = 'https://www.indeed.com/jobs?q={}&radius=35&filter=0&sort=date&start={}'

# Print the pagination URL for Data Analyst
print(pagination_url.format(job_search_keyword[0], 0))


https://www.indeed.com/jobs?q=Data+Analyst&radius=35&filter=0&sort=date&start=0


In [15]:
start = time.time()


job_='Data+Engineer'
location='Washington'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)


driver.get(paginaton_url.format(job_,location,0))

# t = ScrapeThread(url_)
# t.start()

sleep(randint(2, 6))

p=driver.find_element(By.CLASS_NAME,'jobsearch-JobCountAndSortPane-jobCount').text

# Max number of pages for this search! There is a caveat described soon
max_iter_pgs=int(p.split(' ')[0])//15 


driver.quit() # Closing the browser we opened


end = time.time()

print(end - start,'seconds to complete action!')
print('-----------------------')
print('Max Iterable Pages for this search:',max_iter_pgs)

13.331032514572144 seconds to complete action!
-----------------------
Max Iterable Pages for this search: 13


In [19]:
for i in range(0,max_iter_pgs):
    driver.get(paginaton_url.format(job_,location,i*10))
        
    sleep(randint(2, 4))   
    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME,"jobTitle")
        print(job_title.text)

MaxRetryError: HTTPConnectionPool(host='localhost', port=64220): Max retries exceeded with url: /session/edcd36f668d1ceb8ebaf7a92596f866a/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023578A95360>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [11]:
# Pagination: PRACTICE

start = time.time()


job_='Data+Engineer'
location='Washington'


job_lst=[]
job_description_list_href=[]

# job_description_list = []
salary_list=[]


driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                         options=option)
sleep(randint(2, 6))

# driver.get("https://www.indeed.com/q-USA-jobs.html")

for i in range(0,max_iter_pgs):
    driver.get(paginaton_url.format(job_,location,i*10))
    
    
    sleep(randint(2, 4))

    job_page = driver.find_element(By.ID,"mosaic-jobResults")
    jobs = job_page.find_elements(By.CLASS_NAME,"job_seen_beacon") # return a list

    for jj in jobs:
        job_title = jj.find_element(By.CLASS_NAME,"jobTitle")
#         print(job_title.text)
        
# Href's to get full job description (need to re-terate to get full info)
# Reference ID for each job used by indeed         
# Finding the company name        
# Location
# Posting date
# Job description

        job_lst.append([job_title.text,
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href"),
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("id"),      
        jj.find_element(By.CLASS_NAME,"companyName").text,       
        jj.find_element(By.CLASS_NAME,"companyLocation").text,
        jj.find_element(By.CLASS_NAME,"date").text,
        job_title.find_element(By.CSS_SELECTOR,"a").get_attribute("href")])
        

        try: # I removed the metadata attached to this class name to work!
            salary_list.append(jj.find_element(By.CLASS_NAME,"salary-snippet-container").text)

        except NoSuchElementException: 
            try: 
                salary_list.append(jj.find_element(By.CLASS_NAME,"estimated-salary").text)
                
            except NoSuchElementException:
                salary_list.append(None)
      
                
#         # Click the job element to get the description
#         job_title.click()
        
#         # Help to load page so we can find and extract data
#         sleep(randint(3, 5))

#         try: 
#             job_description_list.append(driver.find_element(By.ID,"jobDescriptionText").text)
            
#         except: 
            
#             job_description_list.append(None)

driver.quit() 


end = time.time()

print(end - start,'seconds to complete Query!')

# alternate way to grab the info for job description to make it faster:


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (chrome not reachable)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
	GetHandleVerifier [0x00A46EE3+174339]
	(No symbol) [0x00970A51]
	(No symbol) [0x00686FF6]
	(No symbol) [0x006AA756]
	(No symbol) [0x006A7A0F]
	(No symbol) [0x006DE259]
	(No symbol) [0x006DDBBA]
	(No symbol) [0x006D6DA6]
	(No symbol) [0x006B1034]
	(No symbol) [0x006B1F8D]
	GetHandleVerifier [0x00AE4B1C+820540]
	sqlite3_dbdata_init [0x00BA53EE+653550]
	sqlite3_dbdata_init [0x00BA4E09+652041]
	sqlite3_dbdata_init [0x00B997CC+605388]
	sqlite3_dbdata_init [0x00BA5D9B+656027]
	(No symbol) [0x0097FE6C]
	(No symbol) [0x009783B8]
	(No symbol) [0x009784DD]
	(No symbol) [0x00965818]
	BaseThreadInitThunk [0x76CF7BA9+25]
	RtlInitializeExceptionChain [0x778CBD2B+107]
	RtlClearBits [0x778CBCAF+1

In [16]:
job_lst[0:2]

[]

In [13]:
salary_list[0:3]

[]

In [ ]:
url = "https://in.indeed.com/"